In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_df = pd.read_csv("train-data.tsv",sep='\t', names=['label', 'sms'])
valid_df = pd.read_csv("valid-data.tsv",sep='\t', names=['label', 'sms'])
train_df.head()

In [ ]:
train_df_label = train_df['label'].tolist()
valid_df_label = valid_df['label'].tolist()

for i in range(len(train_df_label)):
  if train_df_label[i] == 'ham':
    train_df_label[i] = 1
  else:
    train_df_label[i] = 0

for i in range(len(valid_df_label)):
  if valid_df_label[i] == 'ham':
    valid_df_label[i] = 1
  else:
    valid_df_label[i] = 0

train_df['label'] = train_df_label
valid_df['label'] = valid_df_label


In [ ]:
# Create model to predict sms
from sklearn.metrics import classification_report, accuracy_score, f1_score , recall_score, roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer()
vectorizer = vectorizer.fit(train_df['sms'])

X_train = vectorizer.transform(train_df['sms'].tolist())
y_train = np.array(train_df['label'].tolist())
X_test = vectorizer.transform(valid_df['sms'].tolist())
y_test = np.array(valid_df['label'].tolist())

clf_lr = LogisticRegression()
clf_lr.fit(X_train, y_train)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  prediction = []
  hash_map = {0: 'ham', 1: "spam"}
  pred_text = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]
  pred_text = vectorizer.transform(pred_text)
  re = clf_lr.predict_proba(pred_text)
  for el in re:
      prediction.append([max(el),hash_map[np.argmax(el)]])


  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
